In [1]:
% matplotlib inline

import numpy as np

from scipy.integrate import quad
from scipy.interpolate import interp1d
from scipy.signal import butter, filtfilt, iirdesign, zpk2tf, freqz
from scipy.special import gammaln
from scipy.stats import chisquare, chi2

from sklearn.cluster import MeanShift, estimate_bandwidth, DBSCAN

import matplotlib.pyplot as plt
import matplotlib.mlab as mlab

import h5py
import pywt

import readligo as rl

In [2]:
# function to whiten data
def whiten(strain, interp_psd, dt):
    Nt = len(strain)
    freqs = np.fft.rfftfreq(Nt, dt)
    
    hf = np.fft.rfft(strain)
    white_hf = hf / (np.sqrt(interp_psd(freqs)/dt/2.))
    white_ht = np.fft.irfft(white_hf, n=Nt)
    return white_ht

In [3]:
fs = 4096 # sampling rate

# get segment list. note that for now, I'm only using a small amount of
# data (1 file), but that this should be expandable to many more (currently
# I have about 50GB worth of data cached on my home desktop, which should
# be more than enough for this)
start = 931035615
stop = 932245215
fl = rl.FileList()
segList = rl.getsegs(start, stop, 'H1', flag='DATA', filelist=fl)
print segList

Using data directory . ...
WARNING!  No file found for GPS 931033088 and IFO H1
WARNING! No file found with GPS start time 931033088
Segment list may contain errors due to missing files.
WARNING!  No file found for GPS 931037184 and IFO H1
WARNING! No file found with GPS start time 931037184
Segment list may contain errors due to missing files.
WARNING!  No file found for GPS 931041280 and IFO H1
WARNING! No file found with GPS start time 931041280
Segment list may contain errors due to missing files.
WARNING!  No file found for GPS 931045376 and IFO H1
WARNING! No file found with GPS start time 931045376
Segment list may contain errors due to missing files.
WARNING!  No file found for GPS 931057664 and IFO H1
WARNING! No file found with GPS start time 931057664
Segment list may contain errors due to missing files.
WARNING!  No file found for GPS 931061760 and IFO H1
WARNING! No file found with GPS start time 931061760
Segment list may contain errors due to missing files.
WARNING!  No 

WARNING!  No file found for GPS 932077568 and IFO H1
WARNING! No file found with GPS start time 932077568
Segment list may contain errors due to missing files.
WARNING!  No file found for GPS 932134912 and IFO H1
WARNING! No file found with GPS start time 932134912
Segment list may contain errors due to missing files.
WARNING!  No file found for GPS 932139008 and IFO H1
WARNING! No file found with GPS start time 932139008
Segment list may contain errors due to missing files.
WARNING!  No file found for GPS 932151296 and IFO H1
WARNING! No file found with GPS start time 932151296
Segment list may contain errors due to missing files.
WARNING!  No file found for GPS 932155392 and IFO H1
WARNING! No file found with GPS start time 932155392
Segment list may contain errors due to missing files.
WARNING!  No file found for GPS 932167680 and IFO H1
WARNING! No file found with GPS start time 932167680
Segment list may contain errors due to missing files.
WARNING!  No file found for GPS 93217996

In [ ]:
# loop over good segments
#fn_H1 = 'cache/H-H1_LOSC_4_V2-1126257414-4096.hdf5' #'cache/H-H1_LOSC_4_V2-1126259446-32.hdf5'
# 'H-H1_LOSC_4_V2-1126257414-4096.hdf5'
#strain, time, chan_dict_H1 = rl.loaddata(fn_H1, 'H1')
#begin = time[0]
#end = time[-1]
#ts = time[1]-time[0]

event_data = np.array([])
for (begin, end) in segList: # for q in [0]:
    strain, meta, dq = rl.getstrain(begin, end, 'H1')
    ts = meta['dt']
    
    # first whiten strain data
    NFFT = 1*fs    
    Pxx, freqs = mlab.psd(strain, Fs=fs, NFFT=NFFT)
    psd = interp1d(freqs, Pxx)
    
#     plt.figure()
#     plt.loglog(freqs, np.sqrt(Pxx),'r',label='H1 strain')
#     plt.axis([10, 2000, 1e-24, 1e-19])
#     plt.grid('on')
#     plt.ylabel('ASD (strain/$\sqrt{Hz}$)')
#     plt.xlabel('Freq (Hz)')
#     plt.title('Advanced LIGO strain data near GW150914')
#     plt.savefig('GW150914_prewhite.png')
    
    strain_whiten = whiten(strain,psd,ts)
    
    Pxx_white, freqs_white = mlab.psd(strain_whiten, Fs=fs, NFFT=NFFT)
    psd_white = interp1d(freqs_white, Pxx_white)
    
#     plt.figure()
#     plt.loglog(freqs_white, np.sqrt(Pxx_white),'r',label='H1 strain')
#     plt.grid('on')
#     plt.ylabel('ASD (strain/$\sqrt{Hz}$)')
#     plt.xlabel('Freq (Hz)')
#     plt.title('Whitened Advanced LIGO strain data near GW150914')
#     plt.savefig('GW150914_white.png')
    
    bb, ab = butter(4, [80.*2./fs, 300.*2./fs],btype='band') #butterworth bandpass
    strain_whitenbp = filtfilt(bb, ab, strain_whiten)
    
    # plot of the whitened data and unwhitened data
    # not really relevant to final goals, but it shows the whitening is working!
#     figure1 = plt.figure(figsize=(10,8))
#     plt.subplot(2,1,1)
#     rel_time = np.arange(begin, end+ts, ts)
    
#     plt.plot(rel_time, strain)
#     plt.title('Raw Timeseries')
#     plt.xlabel('GPS Time($s$)')
    
#     plt.subplot(2,1,2)
#     plt.plot(rel_time, strain_whitenbp)
#     plt.title('Whitened Timeseries')
#     plt.xlabel('GPS Time($s$)')
    
#     plt.tight_layout(pad=1.0, w_pad=1.0, h_pad=1.0)
    
    #------------------------------------------------------------
    
    # then wavelet decomposition.
    # returns [cA_n, cD_n, cD_n-1, ..., cD2, cD1]
    # where n is the decomposition level, and cA are the approximation coefficient arry
    # while cD are details coefficient arrays
    coeffs = pywt.wavedec(strain_whitenbp, 'haar') # using haar for now, as thats what LIGO apparently uses for this. will do more research later to verify reasoning
    D_coeffs = np.flipud(coeffs[1:]) # switching the order so that D_coeffs[i] corresponds to j=i+1 decomp level
    
    E_coeffs = []
    seg_times = []
    all_scales = []
    for scale, D_co in enumerate(D_coeffs):
        N = len(D_co)
        sigma = np.std(D_co)
        if (sigma == 0):
            break
        sig_px = np.where((D_co / sigma) > 4.)[0]
        
        scales = np.full(np.shape(sig_px),scale+1,dtype=np.int64)

        times = np.full(np.shape(sig_px),begin,dtype=np.float64)
        for i in range(len(sig_px)):
            times[i] += ((2.**(scale+1.)) / fs) * sig_px[i]
            
        seg_times = np.append(seg_times, times)
        all_scales = np.append(all_scales,scales).astype(np.int64)

        E_coeffs = np.append(E_coeffs,((D_co**2) /  (np.std(D_co)**2))[sig_px])
    
    data = np.vstack((seg_times, all_scales * ts)).T # multiplying by ts to rescale the scale axis
    
    ms = MeanShift()
    ms.fit(data)
    
    labels = ms.labels_
    centers = ms.cluster_centers_
    
#     fig2 = plt.figure(figsize=(10,8))
#     plt.scatter(seg_times,all_scales,c=labels,cmap=plt.cm.viridis)
#     plt.scatter(centers[:,0],centers[:,1],marker='*',c='r',label='Cluster Centers')
#     plt.legend()
#     plt.title('')
#     plt.xlabel('GPS Time ($s$)')
#     plt.ylabel('$j$')
#     plt.savefig('GW150914_cluster.png')
    
    #------------------------------------------------------------
    for label in np.unique(labels):
        pts = np.where(labels==label)
        clust = E_coeffs[pts]
        E_c = np.sum(clust)
        x = np.linspace(0,1000,1000)
    
        signif = -np.log(quad(chi2.pdf, E_c, np.inf, args=(len(clust),))[0])
        if (signif > 20.):
            if (len(event_times) == 0):
                event_times = np.array([min(seg_times[pts]),max(seg_times[pts])])
            else:
                time_coords = np.array([min(seg_times[pts]),max(seg_times[pts])])
                event_data = np.vstack((event_times,time_coords))
    break
    

np.savetxt('H1_times.csv', event_data, delimiter=' ')